NOMBRES: Fátima Priscilla 

APELLIDOS: González Sandoval

CARNE: 20689

FECHA: 24 de agosto de 2024

**Ejercicio 1**
Con los datos de la clase de lunes, cálcule PPMI, pero aplicando Lapace Smoothing.

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = '../PMI_Ejercicio/pmi_ejercicio.xlsx'
df_PMI = pd.read_excel(file_path)
df_PMI.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Computer,data,result,pie,sugar
1,Cherry,2,8,9,44260,25
2,Strawberry,0,0,1,5,19
3,Digital,1670,1683,85,5,4
4,Information,3325,3982,378,512,13


In [3]:
# Limpia de datos
df_PMI.columns = df_PMI.iloc[0]  
df_PMI = df_PMI.drop(df_PMI.index[0])  
df_PMI = df_PMI.set_index(df_PMI.columns[0]) 

df_PMI = df_PMI.apply(pd.to_numeric, errors='coerce').fillna(0)

# Aplicar suavizado de Laplace sumando 1 a cada cuenta
laplace_df_PMI = df_PMI + 1

total_sum = laplace_df_PMI.sum().sum()

Px = laplace_df_PMI.sum(axis=1) / total_sum  
Py = laplace_df_PMI.sum(axis=0) / total_sum  
Pxy = laplace_df_PMI / total_sum  

pmi = np.log2(Pxy / (Px[:, None] * Py[None, :]))

ppmi = np.maximum(pmi, 0)

# Convertir a DataFrame para facilitar la visualización
ppmi_df_PMI = pd.DataFrame(ppmi, index=laplace_df_PMI.index, columns=laplace_df_PMI.columns)

ppmi_df_PMI.head()

/var/folders/8f/4tqfct110qxbk9qrh28g1lpr0000gn/T/ipykernel_70723/3563902525.py:17: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  pmi = np.log2(Pxy / (Px[:, None] * Py[None, :]))


,Computer,data,result,pie,sugar
nan,,,,,
Cherry,0.000000,0.000000,0.000000,0.32097,0.000000
Strawberry,0.000000,0.000000,2.968558,0.00000,9.165963
Digital,2.438572,2.266840,1.548496,0.00000,0.319637
Information,2.180820,2.257976,2.437457,0.00000,0.554236


**Ejercicio 2**

POC para crear información de entreno

In [4]:
#Librerías que necesitarán
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
sentece = "The wind Crosses the brown land unheard" #De mis poemas fav. The waste land de T.S Elliot
tokens = tf.keras.preprocessing.text.text_to_word_sequence(sentece) #tokenizar
vocab, index = {}, 1 #arreglo vocabulario, empezar indice en 1
vocab["algo importante para que no explote el programa"] = 0

#Llene el arreglo para vocabulario
for word in tokens:
    if word not in vocab:
        vocab[word] = index
        index += 1

vocab_size = len(vocab)
example_sequence = [vocab[word] for word in tokens]
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0
) #rellene esta area como le sea pertinente)

print("Tokens:", tokens)
print("Vocabulario:", vocab)
print("Secuencia de ejemplo:", example_sequence)
print("Positive Skip-Grams:", positive_skip_grams)

Tokens: ['the', 'wind', 'crosses', 'the', 'brown', 'land', 'unheard']
Vocabulario: {'algo importante para que no explote el programa': 0, 'the': 1, 'wind': 2, 'crosses': 3, 'brown': 4, 'land': 5, 'unheard': 6}
Secuencia de ejemplo: [1, 2, 3, 1, 4, 5, 6]
Positive Skip-Grams: [[6, 5], [1, 3], [6, 4], [4, 1], [4, 5], [2, 1], [1, 5], [4, 3], [1, 3], [4, 6], [1, 2], [3, 1], [5, 4], [2, 1], [1, 2], [3, 4], [5, 1], [3, 2], [3, 1], [2, 3], [5, 6], [1, 4]]


In [6]:
#Resultados
inverse_vocab = {index: token for token, index in vocab.items()}
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(6, 5): (unheard, land)
(1, 3): (the, crosses)
(6, 4): (unheard, brown)
(4, 1): (brown, the)
(4, 5): (brown, land)


In [7]:
target_word, context_word = positive_skip_grams[0]
# Escoja un numero para muestras negativos (que no pertenence al contexto)
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # clase que debe ser muestreada como positiva (que pertenece al contexto)
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,  # [0, vocab_size]
    seed=SEED,
    name="negative_sampling"
)

In [8]:
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wind', 'the', 'brown', 'crosses']


In [9]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=vocab_size) #funcion que construye la tabla de muestreo en forma de frecuencias
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381]


In [10]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1) #se agrega una dimension para poder concatenar
context = tf.concat([context_class, negative_sampling_candidates], 0)
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

#Listo! asi se prepara la info para entrenar

In [11]:
#Informacion a utilizar:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [12]:
#Paso 1: cree una funcion que estandarice el texto como ya hemos hecho e incrustela en una capa de Tensorflow (TextVectorization)
def vect(input_text):
    lowercase = tf.strings.lower(input_text)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, f'[{re.escape(string.punctuation)}]', '')

vectorize_layer = layers.TextVectorization(
    standardize=vect,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=50  
)

In [14]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
#Paso 2: Usando la POC de creacion de datos de entreno, defina una funcion para este proceso






In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

In [ ]:
#El siguiente codigo es el modelo Word2Vec usando lo que ya han hecho. Sin embargo, deben agregar la metrica de similitud que vimos en clase, a la cual deben llamar dots
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)


    # dots: (batch, context)
    return dots

In [ ]:
#Entrenar modelo y definir CategoricalCrossEntropy como funcion de perdida. Mostrar el print de accuracy y loss. Si deciden presentar sus resultados en tensorboard, adjuntar capturas.